# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
# from api_keys import geoapify_key (NOT Working Properly)
geoapify_key = "3cf156db96744c979793d50f16093e65"

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,talnakh,69.4865,88.3972,0.67,99,100,2.73,RU,1684710139
1,1,tvoroyri,61.5556,-6.8111,8.64,86,71,10.30,FO,1684710139
2,2,buka,40.8108,69.1986,19.96,31,1,1.92,UZ,1684710140
3,3,edinburgh of the seven seas,-37.0676,-12.3116,15.81,85,82,8.75,SH,1684710140
4,4,waitangi,-43.9535,-176.5597,13.23,75,60,5.36,NZ,1684710140


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [35]:
%%capture --no-display

# Configure the map plot
city_map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    size = 'Humidity',
    color='City')

# Display the map
city_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [36]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_city_df = city_data_df.loc[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) \
                                 & (city_data_df['Wind Speed'] < 4.5) \
                                 & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
narrow_city_df = narrow_city_df.dropna()

# Display sample data
narrow_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,hamilton,39.1834,-84.5333,25.34,41,0,2.57,US,1684709826
44,44,college,64.8569,-147.8028,21.93,41,0,0.45,US,1684710002
154,154,remire-montjoly,4.9167,-52.2667,26.02,89,0,1.54,GF,1684710176
387,387,sabha,27.0377,14.4283,23.57,17,0,2.77,LY,1684710237
494,494,todos santos,23.4500,-110.2167,21.36,73,0,3.75,MX,1684710263
509,509,jeddah,21.5169,39.2192,25.92,53,0,2.06,SA,1684710266
522,522,elk city,35.4120,-99.4043,24.38,46,0,3.09,US,1684710254
538,538,sallisaw,35.4604,-94.7875,23.94,58,0,2.06,US,1684710006
542,542,tete,-16.1564,33.5867,21.84,70,0,1.30,MZ,1684710274


### Step 3: Create a new DataFrame called `hotel_df`.

In [37]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_city_df.copy()
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
21,hamilton,US,39.1834,-84.5333,41,
44,college,US,64.8569,-147.8028,41,
154,remire-montjoly,GF,4.9167,-52.2667,89,
387,sabha,LY,27.0377,14.4283,17,
494,todos santos,MX,23.4500,-110.2167,73,
509,jeddah,SA,21.5169,39.2192,53,
522,elk city,US,35.4120,-99.4043,46,
538,sallisaw,US,35.4604,-94.7875,58,
542,tete,MZ,-16.1564,33.5867,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [38]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'api_key': geoapify_key,
    'categories': 'accommodation.hotel'
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hamilton - nearest hotel: North Vista Manor
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
remire-montjoly - nearest hotel: Complexe Belova
sabha - nearest hotel: فندق الكاظم-سبها
todos santos - nearest hotel: Hotel Casa Tota
jeddah - nearest hotel: No hotel found
elk city - nearest hotel: Hampton
sallisaw - nearest hotel: Quality Inn
tete - nearest hotel: Zambezi Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
21,hamilton,US,39.1834,-84.5333,41,North Vista Manor
44,college,US,64.8569,-147.8028,41,BEST WESTERN PLUS Pioneer Park Inn
154,remire-montjoly,GF,4.9167,-52.2667,89,Complexe Belova
387,sabha,LY,27.0377,14.4283,17,فندق الكاظم-سبها
494,todos santos,MX,23.4500,-110.2167,73,Hotel Casa Tota
509,jeddah,SA,21.5169,39.2192,53,No hotel found
522,elk city,US,35.4120,-99.4043,46,Hampton
538,sallisaw,US,35.4604,-94.7875,58,Quality Inn
542,tete,MZ,-16.1564,33.5867,70,Zambezi Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [39]:
%%capture --no-display

# Configure the map plot
city_map_plot = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo=True,
    tiles='OSM',
    title='City Data Map',
    frame_width=800,
    frame_height=600,
    color='City',
    hover_cols=['City', 'Country', 'Hotel Name'])

# Display the map
city_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)